In [80]:
import netCDF4
from netCDF4 import Dataset

import csv
import pandas as pd

import numpy as np
from numpy import newaxis
# from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt
# from matplotlib.lines import Line2D

import math

import warnings

%matplotlib inline


In [81]:
def ncdump(nc_fid, verb=True):
    
    def print_ncattr(key):
  
        try:
            print(("\t\ttype:", repr(nc_fid.variables[key].dtype)))
            for ncattr in nc_fid.variables[key].ncattrs():
                print(('\t\t%s:' % ncattr,\
                      repr(nc_fid.variables[key].getncattr(ncattr))))
        except KeyError:
            print(("\t\tWARNING: %s does not contain variable attributes" % key))

    # NetCDF global attributes
    nc_attrs = nc_fid.ncattrs()
    if verb:
        print("NetCDF Global Attributes:")
        for nc_attr in nc_attrs:
            print('\t%s:' % nc_attr, repr(nc_fid.getncattr(nc_attr)))
    nc_dims = [dim for dim in nc_fid.dimensions]  # list of nc dimensions
    # Dimension shape information.
    if verb:
        print("NetCDF dimension information:")
        for dim in nc_dims:
            print("\tName:", dim) 
            print("\t\tsize:", len(nc_fid.dimensions[dim]))
            print_ncattr(dim)
    # Variable information.
    nc_vars = [var for var in nc_fid.variables]  # list of nc variables
    if verb:
        print("NetCDF variable information:")
        for var in nc_vars:
            if var not in nc_dims:
                print('\tName:', var)
                print("\t\tdimensions:", nc_fid.variables[var].dimensions)
                print("\t\tsize:", nc_fid.variables[var].size)
                print_ncattr(var)
    return nc_attrs, nc_dims, nc_vars

In [82]:
nc_f = 'SABER_2009-01.nc'
ds = Dataset(nc_f, mode='r')

lat = ds.variables['tplatitude'][:] # dimensions: ('event', 'altitude')
#og_lat = ds.variables['tplatitude'][:] # dimensions: ('event', 'altitude')
lon = ds.variables['tplongitude'][:] # dimensions: ('event', 'altitude')
temp = ds.variables['ktemp'][:] # dimensions: ('event', 'altitude') # in K
alt = ds.variables['tpaltitude'][:] # dimensions: ('event', 'altitude')
date = ds.variables['date'][:] # dimensions: ('event',) # yyyyddd

ds.close()

In [188]:
def get_gridded_data(dx, dy, dz):
    
    #Make Grids
    #Longitude (every dx degree), Latitude (every dy degree), Altitude (every dz km)
    newlon = np.arange(dx/2,360,dx)
    newlat = np.arange(-90+dy/2,90,dy)
    newalt = np.arange(30,100,dz)

    nx=newlon.size
    ny=newlat.size
    nz=newalt.size

    #Make Empty Grid box
    tempgrid=np.zeros((nx,ny,nz))
    
    return newlon, newlat, newalt, tempgrid,nx,ny,nz

def get_dailysaber_data(targd):

    #SABER lon and lat for each profile
    saberlonall = lon[:,200]
    saberlatall = lat[:,200]

    # Find date
    dbin = np.where(date == targd)[0]
        
    # lond, latd, tempd, altd = data for each day
    lond = saberlon[dbin]
    latd = saberlat[dbin]
    tempd = temp[dbin,:]
    altd = alt[dbin,:]

    nprofday = lond.size
    print(nprofday)
    return lond, latd, tempd, altd, nprofday



In [191]:
#Get New Grid
dx = 24
dy = 10
dz = 2 
newlon, newlat, newalt, tempgrid, nx, ny, nz = get_gridded_data(dx, dy, dz)

#Get SABER Data
dateuse = 2009015
lond, latd, tempd, altd, nprofday= get_dailysaber_data(dateuse)

for i in range(nx):
    lonbin = np.where((lond > newlon[i]-dx/2.) & (lond < newlon[i]+dx/2.))[0]
    nlonb =lonbin.size
        
    if nlonb > 0:
        lat1 = latd[lonbin]
        lon1 = lond[lonbin]
        temp1 = tempd[lonbin,:]
        alt1 = altd[lonbin,:]

        for j in range(ny):
            latbin = np.where((lat1 > newlat[j]-dy/2.) & (lat1 < newlat[j]+dy/2.))[0]
            nla =latbin.size

            if nla > 0:
                lat2 = lat1[latbin]
                lon2 = lon1[latbin]
                temp2 = temp1[latbin,:]
                alt2 = alt1[latbin,:]



1432


In [187]:
#Get New Grid
dx = 24
dy = 10
dz = 2 
newlon, newlat, newalt, tempgrid, nx, ny, nz = get_gridded_data(dx, dy, dz)

print(ny)
#Get SABER Data
dateuse = 2009015
lond, latd, tempd, altd, nprofday= get_dailysaber_data(dateuse)

#def get_gridded_altitude(tempd,altd, newalt,dz):

nprof=lond.size
    
for pr in range(nprof):
    tempu = tempd(pr,:)
    altu = altd(pr,:)
    for k in range(nz):
        altbin = np.where((altu > newalt[k]-dz/2.) & (altu < newalt[k]+dz/2.))[0]
        tempz[pr,k]=np.mean(tempu[altbin])

#    return tempz

SyntaxError: invalid syntax (<ipython-input-187-e83d7ea6fc60>, line 17)

In [186]:
print(nprofday)

1432
